### Scraping USA Snow Storm Data and visualizing them using Folium
### Samira Kumar
### Data Source: https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00464

In [5]:
import re
from bs4 import BeautifulSoup
import geopandas as gpd
import requests
from pathlib import Path
import gzip
import glob
import os.path
from pathlib import Path
import pandas
import urllib

In [3]:
r  = requests.get("https://www.ncei.noaa.gov/data/snowstorm-database/archive/")
data = r.text
soup = BeautifulSoup(data)

link_a=[]
for link in soup.find_all('a'):
    link_a.append(link.get('href'))
link_a[0:10]

['?C=N;O=D',
 '?C=M;O=A',
 '?C=S;O=A',
 '?C=D;O=A',
 '/data/snowstorm-database/',
 'snowstorm-db_shapefile-readme_c20130529.txt',
 'snowstorm-db_storm-1_s19000226_e19000303_c20130514.tar.gz',
 'snowstorm-db_storm-1_s19000304_e19000307_c20130514.tar.gz',
 'snowstorm-db_storm-1_s19000315_e19000316_c20130514.tar.gz',
 'snowstorm-db_storm-1_s19010201_e19010206_c20130514.tar.gz']

#### Download all files from the link
**Links** in **link_a** start from 6

In [6]:
testfile = urllib.request.URLopener()
for a in link_a[6:]:
    testfile.retrieve("https://www.ncei.noaa.gov/data/snowstorm-database/archive/"+a, "Data_Script/"+a)

In [ ]:
folder = Path("Path to all unzipped folder")
folder_compiled = Path("Path to store compiled shapefile")


shapefiles = folder.glob("snowstorm-db_*/*.shp")
gdf_1 = pandas.concat([
    gpd.read_file(shp)
    for shp in shapefiles
]).pipe(gpd.GeoDataFrame)
gdf_1.head()

In [ ]:
shapefiles = folder.glob("RSI_*/*/*.shp")
gdf_2 = pandas.concat([
    geopandas.read_file(shp)
    for shp in shapefiles
]).pipe(geopandas.GeoDataFrame)
gdf_2.head()

#### We're using 2 separate geodataframes since the structure of folders of the data is different in old set of years. 

In [ ]:
gdf_concat=pandas.concat([gdf_1,gdf_2])
gdf_concat.to_file(folder_compiled / 'compiled.shp')

#### I'd save this file in both shapfile and csv, but used only the csv to prepare the visualisation

## Visualization using Folium

In [8]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
import numpy as np
import os
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata,RectSphereBivariateSpline,Rbf
import geojsoncontour
import scipy as sp
import scipy.ndimage 
import json

In [ ]:
df=pd.read_csv('filtered_data.csv')
df.head()

In [ ]:
snowfall_cat=[]
for i,v in df.iterrows():
    sf=v['Total Snowfall']
    if sf<2:
        snowfall_cat.append(0)
    elif (sf>=2) and (sf<6):
        snowfall_cat.append(0.2)
    elif (sf>=6) and (sf<12):
        snowfall_cat.append(0.4)
    elif (sf>=12) and (sf<18):
        snowfall_cat.append(0.6)
    elif (sf>=18) and (sf<24):
        snowfall_cat.append(0.8)
    else:
        snowfall_cat.append(1)
df['Snowfall Cat']=snowfall_cat
df.head()

In [ ]:
df=df[~(df['Latitude']<=0)].copy()
df.head()

### The below method was initially taken from Thomas Jansson's post (https://github.com/python-visualization/folium/issues/958#issuecomment-427156672) and modified to create the map for time series data.

#### This creates the contour plot for the map

In [ ]:
# Iterate all years in the dataset
data = []
a={}
time_not_in=[1911,1928] #these 2 years dont have data. So ignore them while checking
min_year,max_year=int(df.Year_Start.min()),int(df.Year_Start.max())
for i in range(1900,max_year+1):
    if (i not in time_not_in):
        
        df_copy=df[df['Year_Start']==i].copy()
        colors = ['white',  'royalblue',  'cyan',  'lime',  'yellow','red']
        vmin   = 0
        vmax   = 1
        levels = len(colors)
        cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

        
        x_orig = (df_copy.Longitude.values.tolist())
        y_orig = (df_copy.Latitude.values.tolist())
        z_orig = np.asarray(df_copy['Snowfall Cat'].values.tolist())

        
        x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), 500)
        y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), 500)
        x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)

        xscale = df_copy.Longitude.max() - df_copy.Longitude.min()
        yscale = df_copy.Latitude.max() - df_copy.Latitude.min()

        scale = np.array([xscale, yscale])

        
        z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')

        
        sigma = [3, 3]
        z_mesh = sp.ndimage.filters.gaussian_filter(z_mesh, sigma, mode='nearest')

        # Create the contour
        contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, 
                                linestyles='none', vmin=vmin, vmax=vmax)

        # Convert matplotlib contourf to geojson
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contourf,
            min_angle_deg=3.0,
            ndigits=3,
            unit='m',
            stroke_width=1,
            fill_opacity=0.3)
        d = json.loads(geojson)
        len_features=len(d['features'])
        for j in range(0,len_features):
            d['features'][j]['properties']['time']=str(i)+'-01-31'
        if not data:
            data.append(d)
        else:
            for i in range(len(d['features'])):
                 data[0]['features'].append(d['features'][i])

print ("Completed!!")

### The below code modifies the geojson to change properties of styling.

In [ ]:
dict_act=data[0]
for i in range(len(dict_act['features'])):

    dict_act['features'][i]['properties']['color']=dict_act['features'][i]['properties'].pop('fill')
    dict_act['features'][i]['properties']['weight']=dict_act['features'][i]['properties'].pop('stroke-width')
    dict_act['features'][i]['properties']['fillOpacity']=dict_act['features'][i]['properties'].pop('fill-opacity')
    properties=dict_act['features'][i]['properties']
    gettime=properties.pop('time')
    new_properties = {'style': properties, 'time':gettime}
    dict_act['features'][i]['properties']=new_properties
    
print('Completed!!')

In [ ]:
geomap = folium.Map([39, -99], zoom_start=4.5, tiles="cartodbpositron")

#{'type': 'FeatureCollection',
 #       'features': features}
TimestampedGeoJson(
    dict_act,
    period='P1Y',
    duration='P1M',
    auto_play=False,
    date_options='YYYY'
    ).add_to(geomap)#date_options='YYYY'

# Add the colormap to the folium map
cm.caption = 'Snowfall Category'
geomap.add_child(cm)

# Fullscreen mode
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)

# Plot the data
geomap.save('folium_contour_time.html')

print('Completed!!')

### Final Visualisation:
## https://samirak93.github.io/snow/index.html